First we set up the notebook and import our tools.  I will run this book on kernal PUI2016_Python3

In [2]:
from __future__ import print_function, division
import geopandas as gp
import pandas as pd
import pylab as pl
import os
import json
import seaborn as sns
import matplotlib as plt
import numpy as np
import sys

#s = json.load( open(os.getenv('PUI2016')+'/fbb_matplotlibrc.json') )
#pl.rcParams.update(s)
if os.getenv("PUIDATA") is None:
    print ("$PUIDATA to point to set PUIdata dir")

%pylab inline
print(os.getenv("PUIDATA"))

Populating the interactive namespace from numpy and matplotlib
/home/cusp/cjs676/PUIdata


/opt/rh/anaconda/root/envs/PUI2016_Python3/lib/python3.5/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)
/opt/rh/anaconda/root/envs/PUI2016_Python3/lib/python3.5/site-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['plt']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


Below is the function which will get and move the citibike data

In [3]:


#check if the env variable PUIDATA exists
if os.getenv("PUIDATA") is None:
    print ("must set PUIDATA env variable")
    sys.exit()

def getCitiBikeCSV(datestring):
    '''Downloads citibike data and unzips it. If the data is downloaded by not unzippeds it zips it. Moves the data to $PUIDATA
    Arguments:
        date string as yyyymm
    '''
    print ("Downloading", datestring)
    ### First I will heck that it is not already there
    if not os.path.isfile(os.getenv("PUIDATA") + "/" + datestring + "-citibike-tripdata.csv"):
        if os.path.isfile(datestring + "-citibike-tripdata.csv"):
            # if in the current dir just move it
            if os.system("mv " + datestring + "-citibike-tripdata.csv " + os.getenv("PUIDATA")):
                print ("Error moving file!, Please check!")
        #otherwise start looking for the zip file
        else:
            if not os.path.isfile(os.getenv("PUIDATA") + "/" + datestring + "-citibike-tripdata.zip"):
                if not os.path.isfile(datestring + "-citibike-tripdata.zip"):
                    os.system("curl -O https://s3.amazonaws.com/tripdata/" + datestring + "-citibike-tripdata.zip")
                ###  To move it I use the os.system() functions to run bash commands with arguments
                os.system("mv " + datestring + "-citibike-tripdata.zip " + os.getenv("PUIDATA"))
            ### unzip the csv 
            os.system("unzip " + os.getenv("PUIDATA") + "/" + datestring + "-citibike-tripdata.zip")
            ## NOTE: old csv citibike data had a different name structure. 
            if '2014' in datestring:
                os.system("mv " + datestring[:4] + '-' +  datestring[4:] + 
                          "\ -\ Citi\ Bike\ trip\ data.csv " + datestring + "-citibike-tripdata.csv")
            os.system("mv " + datestring + "-citibike-tripdata.csv " + os.getenv("PUIDATA"))
    ### One final check:
    if not os.path.isfile(os.getenv("PUIDATA") + "/" + datestring + "-citibike-tripdata.csv"):
        print ("WARNING!!! something is wrong: the file is not there!")

    else:
        print ("file in place, you can continue")


Now we will get the data 201506 first #edit:  success, checked with the Terminal

In [4]:
datestring = '201506'
getCitiBikeCSV(datestring)

file in place, you can continue


###### Now we will get the 201601 data #edit success, checked with the Terminal


In [5]:
datestring = '201601'
getCitiBikeCSV(datestring)

file in place, you can continue


Now we will load the citibike data into datframes
cb15 = 201506 data
cb16 = 201601 data

In [5]:
cb15 = pd.read_csv((os.getenv("PUIDATA") + "/" + '201506' + "-citibike-tripdata.csv"))

#The line line was to test if the path was proper. It is now commented out
#print((os.getenv("PUIDATA") + "/" + '201506' + "-citibike-tripdata.csv"))

In [6]:
#Check to see if the file was read
cb15.head(5)

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,1338,6/1/2015 0:00,6/1/2015 0:22,128,MacDougal St & Prince St,40.727103,-74.002971,2021,W 45 St & 8 Ave,40.759291,-73.988597,20721,Subscriber,1984.0,1
1,290,6/1/2015 0:00,6/1/2015 0:05,438,St Marks Pl & 1 Ave,40.727791,-73.985649,312,Allen St & E Houston St,40.722055,-73.989111,21606,Subscriber,1997.0,1
2,634,6/1/2015 0:01,6/1/2015 0:11,383,Greenwich Ave & Charles St,40.735238,-74.000271,388,W 26 St & 10 Ave,40.749718,-74.002950,16595,Subscriber,1993.0,1
3,159,6/1/2015 0:01,6/1/2015 0:04,361,Allen St & Hester St,40.716059,-73.991908,531,Forsyth St & Broome St,40.718939,-73.992663,16949,Subscriber,1981.0,1
4,1233,6/1/2015 0:02,6/1/2015 0:22,382,University Pl & E 14 St,40.734927,-73.992005,532,S 5 Pl & S 4 St,40.710451,-73.960876,17028,Customer,NaN,0


Repeating the process for the next file.

In [7]:
cb16 = pd.read_csv((os.getenv("PUIDATA") + "/" + '201601' + "-citibike-tripdata.csv"))

#The line line was to test if the path was proper. It is now commented out
#print((os.getenv("PUIDATA") + "/" + '201601' + "-citibike-tripdata.csv"))

In [8]:
cb16.head(5)


,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,923,1/1/2016 00:00:41,1/1/2016 00:16:04,268,Howard St & Centre St,40.719105,-73.999733,3002,South End Ave & Liberty St,40.711512,-74.015756,22285,Subscriber,1958.0,1
1,379,1/1/2016 00:00:45,1/1/2016 00:07:04,476,E 31 St & 3 Ave,40.743943,-73.979661,498,Broadway & W 32 St,40.748549,-73.988084,17827,Subscriber,1969.0,1
2,589,1/1/2016 00:00:48,1/1/2016 00:10:37,489,10 Ave & W 28 St,40.750664,-74.001768,284,Greenwich Ave & 8 Ave,40.739017,-74.002638,21997,Subscriber,1982.0,2
3,889,1/1/2016 00:01:06,1/1/2016 00:15:56,268,Howard St & Centre St,40.719105,-73.999733,3002,South End Ave & Liberty St,40.711512,-74.015756,22794,Subscriber,1961.0,2
4,1480,1/1/2016 00:01:12,1/1/2016 00:25:52,2006,Central Park S & 6 Ave,40.765909,-73.976342,2006,Central Park S & 6 Ave,40.765909,-73.976342,14562,Subscriber,1952.0,1


#Success.  
Now to try to get the IRS data. 
From investigation on the IRS page: File 14zp33ny.xls = https://www.irs.gov/pub/irs-soi/14zp33ny.xls


#First attempt.  Trying to use the hint =on the midterm = ver1. I didnt know where the file was exactly

This time I will use curl and move it

#datestring = '201601'
#getCitiBikeCSV(datestring)

def readIncomeXls(url):
    return pd.read_excel(url, header=3, index_col="ZIP\ncode [1]")

In [9]:
#url = https://www.irs.gov/pub/irs-soi/14zp33ny.xls
os.system('curl -O https://www.irs.gov/pub/irs-soi/14zp33ny.xls')

0

In [10]:
mv 14zp33ny.xls $PUIDATA

mv: setting attribute `security.selinux' for `security.selinux': Permission denied


In [14]:
irs = pd.read_excel((os.getenv("PUIDATA") + "/" + '14zp33ny.xls'), header=3, index_col="ZIP\ncode [1]")
#14zp33ny.xls

#print(os.getenv("PUIDATA") + "/" + '14zp33ny.xls')

Check to see if the file was aquired

In [15]:
irs.head(5)

,Size of adjusted gross income,Number of returns,Number of single returns,Number of joint returns,Number of head of household returns,Number with paid preparer's signature,Number of exemptions,Number of dependents,Number of volunteer prepared returns [2],Unnamed: 10,...,Total tax liability [9],Unnamed: 116,Additional Medicare tax,Unnamed: 118,Net investment income tax,Unnamed: 120,Tax due at time of filing [10],Unnamed: 122,Overpayments refunded [11],Unnamed: 124
ZIP code [1],,,,,,,,,,,,,,,,,,,,,
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Total,Number of volunteer income tax assistance (VIT...,...,Number of returns,Amount,Number of returns,Amount,Number of returns,Amount,Number of returns,Amount,Number of returns,Amount
NaN,NaN,-1.0,-2.0,-3.0,-4.0,-5.0,-6.0,-7.0,-8,-9,...,-114,-115,-116,-117,-118,-119,-120,-121,-122,-123
0,Total,9397410.0,4750550.0,2942890.0,1515950.0,6110600.0,17322490.0,5539120.0,279510,197150,...,7229610,129761061,280780,900175,300410,2745701,1781540,10177242,7115810,25052463
0,"$1 under $25,000",3617400.0,2432240.0,468500.0,665410.0,2247810.0,5220510.0,1678560.0,187260,136800,...,1804270,1677200,0,0,0,0,451080,361017,2886740,6049923
0,"$25,000 under $50,000",2070870.0,1103290.0,449390.0,467540.0,1280840.0,3729240.0,1219770.0,69390,49620,...,1769800,4790944,0,0,0,0,287080,461908,1747120,4510472


In [22]:
#reset the index so that I can use the zipcode column
irs = irs.reset_index()

In [23]:
irs.columns


Index(['ZIP\ncode [1]', 'Size of adjusted gross income', 'Number of returns',
       'Number of single returns', 'Number of joint returns',
       'Number of head of household returns',
       'Number with paid preparer's signature', 'Number of exemptions',
       'Number of dependents', 'Number of volunteer prepared returns [2]',
       ...
       'Total tax liability [9]', 'Unnamed: 116', 'Additional Medicare tax',
       'Unnamed: 118', 'Net investment income tax', 'Unnamed: 120',
       'Tax due at time of filing [10]', 'Unnamed: 122',
       'Overpayments refunded [11]', 'Unnamed: 124'],
      dtype='object', length=125)

In [51]:
irs['ZIP\ncode [1]'] = pd.to_numeric(irs['ZIP\ncode [1]'], errors='coerce')

In [ ]:
IRS

In [41]:
zipcs = pd.to_numeric(irs["ZIP\ncode [1]"], errors='coerce')


In [42]:
print(zipcs)

0            NaN
1            NaN
2            0.0
3            0.0
4            0.0
5            0.0
6            0.0
7            0.0
8            0.0
9            NaN
10       10001.0
11       10001.0
12       10001.0
13       10001.0
14       10001.0
15       10001.0
16       10001.0
17           NaN
18       10002.0
19       10002.0
20       10002.0
21       10002.0
22       10002.0
23       10002.0
24       10002.0
25           NaN
26       10003.0
27       10003.0
28       10003.0
29       10003.0
          ...   
12339    14905.0
12340    14905.0
12341    14905.0
12342    14905.0
12343    14905.0
12344    14905.0
12345        NaN
12346    99999.0
12347    99999.0
12348    99999.0
12349    99999.0
12350    99999.0
12351    99999.0
12352    99999.0
12353        NaN
12354        NaN
12355        NaN
12356        NaN
12357        NaN
12358        NaN
12359        NaN
12360        NaN
12361        NaN
12362        NaN
12363        NaN
12364        NaN
12365        NaN
12366        N

In [43]:
zipcs = zipcs[~np.isnan(zipcs)].astype(int)

In [44]:
print (zipcs)


2            0
3            0
4            0
5            0
6            0
7            0
8            0
10       10001
11       10001
12       10001
13       10001
14       10001
15       10001
16       10001
18       10002
19       10002
20       10002
21       10002
22       10002
23       10002
24       10002
26       10003
27       10003
28       10003
29       10003
30       10003
31       10003
32       10003
34       10004
35       10004
         ...  
12319    14901
12320    14901
12322    14903
12323    14903
12324    14903
12325    14903
12326    14903
12327    14903
12328    14903
12330    14904
12331    14904
12332    14904
12333    14904
12334    14904
12335    14904
12336    14904
12338    14905
12339    14905
12340    14905
12341    14905
12342    14905
12343    14905
12344    14905
12346    99999
12347    99999
12348    99999
12349    99999
12350    99999
12351    99999
12352    99999
Name: ZIP\ncode [1], dtype: int64


In [45]:
list(set(zipcs))


[0,
 99999,
 10001,
 10002,
 10003,
 10004,
 10005,
 10006,
 10007,
 10009,
 10010,
 10011,
 10012,
 10013,
 10014,
 10016,
 10017,
 10018,
 10019,
 10021,
 10022,
 10023,
 10024,
 10025,
 10026,
 10027,
 10028,
 10029,
 10030,
 10031,
 10032,
 10033,
 10034,
 10035,
 10036,
 10037,
 10038,
 10039,
 10040,
 10044,
 10065,
 10069,
 10075,
 10128,
 10162,
 10280,
 10282,
 10301,
 10302,
 10303,
 10304,
 10305,
 10306,
 10307,
 10308,
 10309,
 10310,
 10312,
 10314,
 10451,
 10452,
 10453,
 10454,
 10455,
 10456,
 10457,
 10458,
 10459,
 10460,
 10461,
 10462,
 10463,
 10464,
 10465,
 10466,
 10467,
 10468,
 10469,
 10470,
 10471,
 10472,
 10473,
 10474,
 10475,
 10501,
 10502,
 10504,
 10505,
 10506,
 10507,
 10509,
 10510,
 10511,
 10512,
 10514,
 10516,
 10518,
 10520,
 10522,
 10523,
 10524,
 10526,
 10527,
 10528,
 10530,
 10532,
 10533,
 10535,
 10536,
 10537,
 10538,
 10541,
 10543,
 10546,
 10547,
 10548,
 10549,
 10550,
 10552,
 10553,
 10560,
 10562,
 10566,
 10567,
 10570,
 105

In [49]:
zipcs.head(5)



2    0
3    0
4    0
5    0
6    0
Name: ZIP\ncode [1], dtype: int64